<h1>Utilizing Dask Library to Read Large Datasets</h1>

In [17]:
#Importing dependances and libraries
import dask.dataframe as dd
from dask.delayed import delayed
import dask
import pandas as pd
#from dask.distributed import LocalCluster

In [18]:
#Fully-featured Dask cluster on your local machine
#cluster = LocalCluster()
#client = cluster.get_client()

In [23]:
#Give  variable to the csv files
file1 = "Resources/main_category.csv.gz"
file2 = "Resources/month.csv.gz"
file3 = "Resources/sub_category.csv.gz"

In [25]:
#Creating a list of file names
filenames=[file1,file2,file3]

#Delay reading of files using dask library
for i, fn in enumerate(filenames):
    table = dd.read_csv(fn,compression='gzip',
                        #Change type to Float64
                        dtype={'SUB_CATEGORY':'float64'},
                        blocksize=None)
    if i == 0:
         merged = table
    else:
        merged = dd.merge(merged, table, how="left")

#dir(merged)
merged.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 3 entries, SUB_CATEGORY to MAIN_CATEGORY
dtypes: object(1), float64(1), int64(1)

In [26]:
#clean the merged dataset by dropping N/A from Main_Category
cleaned_ddf = merged.dropna(subset=['MAIN_CATEGORY'])

#cleaned_ddf.astype('float')

#Split large dataset into partitions in for low memory usage
cleaned_ddf = cleaned_ddf.repartition(npartitions=100000)

#Remove spaces in entry
cleaned_ddf['SUB_CATEGORY'] = str(cleaned_ddf['SUB_CATEGORY']).replace(' ','')

#View dataset
#dir(cleaned_ddf['SUB_CATEGORY']) #.info()
cleaned_ddf['SUB_CATEGORY']

Dask Series Structure:
npartitions=100000
    object
       ...
     ...  
       ...
       ...
Name: SUB_CATEGORY, dtype: object
Dask Name: getitem, 9 graph layers

In [27]:
#Iterrate through dataset to collect data and convert to sql
for row in cleaned_ddf.iterrows():
    column_values.add(cleaned_ddf[row])
    #If using Client scheduler
    #sql = client.persist(column_values)
    
    #W/O Client scheduler
    sql = column_values.compute()
    sql = sql.to_sql()
    sql = str(sql)

#Save file to sql
with open('Dataset.sql','w') as file:
    file.write(sql)

ValueError: could not convert string to float: 'Staples - Paper'

<h1>Alternatives to UnZipping CSV.GZ files in Python</h1>

In [ ]:
#Import Dependences
#import gzip
#import pandas as pd
#import csv
#import zipp

###########################################################
#Using the library gzip to open the compressed csv.gz file
###########################################################
#Takes up too much memory and closes program
#with gzip.open(file1, "rt", newline="", encoding="utf-8") as f:
    #reader_file1 = csv.reader(f)

###########################################################
#Method for opening Zip files
###########################################################
#with zipfile.ZipFile(file2) as z:
 #   with z.open(file2_csv) as f:
  #      subcategory = pd.read_csv(f,header=0,delimiter="\t")
   #     print(subcategory.head())

###########################################################
#Using Pandas to unzip files
###########################################################
#subcategory_df = pd.read_csv(file1,compression='gzip',header=0,sep='\t',quotechar='"')
#month_df = pd.read_csv(file2,compression='gzip',header=0,sep='\t',quotechar='"')
#category_df = pd.read_csv(file3,compression='gzip',header=0,sep='\t',quotechar='"')

#Merging tables
#month_cat_subcat = pd.concat([month_df,subcategory_df,category_df],ignore_index=True, sort=False)

#Convert Table to DataFrame
#month_cat_subcat_db = pd.DataFrame(month_cat_subcat)

#Drop NaN values in rows
#month_cat_subcat_db = month_cat_subcat_db.dropna()
#month_cat_subcat_db